In [7]:
pip install pandas seaborn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import json
import pandas as pd

# 定义根目录
root_dir = "output/xbox_infer"

# 用于存储整理后的数据
data = []



# 遍历根目录下的所有文件夹
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    
    # 检查是否是文件夹
    if os.path.isdir(folder_path):
        # 定位all_metrics.jsonl文件
        metrics_file = os.path.join(folder_path, "all_metrics.jsonl")
        
        # 读取jsonl文件
        if os.path.exists(metrics_file):
            with open(metrics_file, 'r') as f:
                # 将metrics按模型和任务类型进行存储
                for line in f:
                    metric_data = json.loads(line)
                    model = folder  # 以文件夹名称作为模型名称
                    task_name = metric_data.get("task_name", "unknown_task")
                    metrics = metric_data.get("metrics", {})
                    for metric_name, metric_value in metrics.items():
                        if metric_name =="hit@5":
                            data.append([model, task_name, metric_name, metric_value])
                        if task_name in ["query2item", "sparse_query2item"]:
                            if metric_name =="coverage@5":
                                data.append([model, task_name, metric_name, metric_value])


# 创建DataFrame
df = pd.DataFrame(data, columns=["model", "task", "metric", "value"])

# 保存为CSV文件
output_csv_path = "output/all_models_metrics_filtered.csv"
df.to_csv(output_csv_path, index=False)

df.head()


,model,task,metric,value
0,ep1,user2item,hit@5,0.168140
1,ep1,gpt_summary,hit@5,0.175385
2,ep1,query2item,coverage@5,0.229350
3,ep1,sparse_query2item,coverage@5,0.335978
4,ep1,title2item,hit@5,0.994792


In [25]:
import os
import json  
import pandas as pd

root_dir = "/home/aiscuser/RecAI/RecLM-emb/data/xbox"
tasks_count = []

# 遍历 train 和 test 目录，读取每个任务的 jsonl 文件条数
for subfolder in ["train", "test"]:
    subfolder_path = os.path.join(root_dir, subfolder)
    if os.path.isdir(subfolder_path):
        for jsonl_file in os.listdir(subfolder_path):
            if jsonl_file.endswith(".jsonl"):
                file_path = os.path.join(subfolder_path, jsonl_file)
                with open(file_path, 'r') as f:
                    line_count = sum(1 for line in f)
                tasks_count.append([subfolder, jsonl_file.replace(".jsonl", ""), line_count])

# 将任务计数数据转换为 DataFrame
tasks_df = pd.DataFrame(tasks_count, columns=["subset", "task", "line_count"])
df = pd.read_csv("/home/aiscuser/RecAI/RecLM-emb/output/all_models_metrics_filtered.csv")

merged_df = pd.merge(df, tasks_df, how="left", on="task")

# 输出到 CSV 文件
output_csv_path = "output/merged_models_metrics.csv"
merged_df.to_csv(output_csv_path, index=False)

# 展示前几行数据
merged_df.head(15)


,model,task,metric,value,subset,line_count
0,ep1,user2item,hit@5,0.168140,train,50759
1,ep1,user2item,hit@5,0.168140,test,50000
2,ep1,gpt_summary,hit@5,0.175385,test,650
3,ep1,query2item,coverage@5,0.229350,train,1152
4,ep1,query2item,coverage@5,0.229350,test,1138
5,ep1,sparse_query2item,coverage@5,0.335978,test,1084
6,ep1,title2item,hit@5,0.994792,train,2734
7,ep1,title2item,hit@5,0.994792,test,384
8,ep1,item2item,hit@5,0.843227,train,892
9,ep1,item2item,hit@5,0.843227,test,657


In [26]:
# 使用 pivot_table 将数据转换为所需格式
pivot_df = merged_df.pivot_table(index='task', columns='subset', values='line_count', aggfunc='sum').reset_index()
pivot_df.rename(columns={'train': 'train_count', 'test': 'test_count'}, inplace=True)

# 遍历唯一的模型名称，创建每个模型的评分列
models = df['model'].unique()

# 用于存储每个模型的评分数据
model_scores_list = []

for model in models:
    model_scores = df[df['model'] == model][['task', 'metric', 'value']].drop_duplicates()

    # 保持 metric 列用于后续合并
    model_scores_pivot = model_scores.pivot_table(index=['task', 'metric'], values='value').reset_index()
    model_scores_pivot.columns = ['task', 'metric', f"{model}"]

    # 将每个模型的评分数据加入列表中
    model_scores_list.append(model_scores_pivot)

# 将所有模型的评分数据逐步合并到一个数据框中
merged_scores_df = model_scores_list[0]
for model_scores in model_scores_list[1:]:
    merged_scores_df = pd.merge(merged_scores_df, model_scores, on=['task', 'metric'], how='left')

# 将行数数据与模型评分合并
final_df = pd.merge(pivot_df, merged_scores_df, on='task', how='left')

# 保存为 CSV 文件
output_csv_path = "output/processed_metrics_summary.csv"
final_df.to_csv(output_csv_path, index=False)



In [27]:
final_df[final_df['task'].isin(['gpt_summary', 'user2item'])]

,task,test_count,train_count,metric,ep1,bge_m3_qwen_v1,ep2,ep3,base,ep4
2,gpt_summary,3900.0,NaN,hit@5,0.175385,0.184615,0.193846,0.206154,0.018462,0.209231
12,user2item,300000.0,304554.0,hit@5,0.168140,0.177900,0.181300,0.197260,0.017780,0.190520


In [14]:
final_df[["task","metric","bge-m3-base","bge-m3_v4"]] # ,"test_count","train_count"

KeyError: "['bge-m3-base', 'bge-m3_v4'] not in index"

In [20]:
import json_repair
conttt = """[{\'title\': \'ReCore\', \'tags\': [\'Action\', \'Action - Science Fiction\', \'Third Person Perspective\', \'Realtime Combat\', \'Post-apocalyptic\', \'Singleplayer\', \'Exploration\', \'Platformer\', \'Stylized Art\', \'Science Fiction\', \'Alien World\', \'Weapons Combat\', \'Ranged Combat\', \'Branching Skill Progression\', \'Collecting\'], \'publisher\': \'Xbox Game Studios\', \'developer\': \'Comcept, Armature Studio\', \'primary genre\': \'Platformer\', \'subgenre\': \'Action and Adventure\', \'devices\': [\'PC\', \'XboxOne\', \'XboxSeries\'], \'franchise\': \'ReCore\', \'release date\': \'September 13, 2016\'}, {\'title\': \'Spiritfarer\', \'tags\': [\'Resource Management\', \'Crafting\', \'Exploration\', \'Simulation\', \'Emotional\', \'Simulation - Life and Romance\', \'Cozy\', \'Hand Drawn Art\', \'Stylized Art\', \'Singleplayer\', \'Character Creation\', \'Atmospheric\', \'Multiplayer Local Cooperative\', \'Side Scroll Perspective\', \'Low Control Complexity\'], \'publisher\': \'Thunder Lotus Games\', \'developer\': \'Thunder Lotus\', \'primary genre\': \'Action and Adventure\', \'subgenre\': \'Action and Adventure\', \'devices\': [\'PC\', \'XboxOne\', \'XboxSeries\'], \'franchise\': \'Spiritfarer\', \'release date\': \'August 18, 2020\'}, {\'title\': \'Open Roads\', \'tags\': [\'Interactive Fiction\', \'Singleplayer\', \'Mystery & Detective\', \'Exploration\', \'Emotional\', \'Choices Matter\', \'Atmospheric\', \'Immersion - Story\', \'Real World\', \'Hand Drawn Art\', \'Stylized Art\', \'Human Experience\', \'High Story Quality\', \'First Person Perspective\', \'High Quality Soundtrack\'], \'publisher\': \'Annapurna Interactive\', \'developer\': \'Open Roads Team\', \'primary genre\': \'Action and Adventure\', \'subgenre\': \'General Adventure\', \'devices\': [\'PC\', \'XboxOne\', \'XboxSeries\'], \'franchise\': \'Open Roads\', \'release date\': \'March 28, 2024\'}, {\'title\': \'Common Hood\', \'tags\': [\'Simulation\', \'Strategy\', \'Resource Management\', \'Realtime\', \'Simulation - Builders\', \'Isometric\', \'Simulation - Realism\', \'Crafting\', \'High Learning Curve\', \'Character Creation\', \'Singleplayer\', \'Real World\', \'Stealth\', \'3D\'], \'publisher\': \'Plethora Project\', \'developer\': \'Plethora Project\', \'primary genre\': \'Simulation\', \'subgenre\': \'Other Games\', \'devices\': [\'PC\', \'XboxOne\', \'XboxSeries\'], \'franchise\': \'Common Hood\', \'release date\': \'July 10, 2023\'}, {\'title\': \'Redfall\', \'tags\': [\'First Person Perspective\', \'Realtime Combat\', \'Open World\', \'Multiplayer\', \'Singleplayer\', \'Weapons Combat\', \'Ranged Combat\', \'Character Creation\', \'Branching Skill Progression\', \'Superpowers\', \'Post-apocalyptic\', \'Action - Science Fiction\', \'Action\', \'Exploration\', \'Survival\'], \'publisher\': \'Xbox Game Studios\', \'developer\': \'Arkane Studios\', \'primary genre\': \'Shooter\', \'subgenre\': \'Action and Adventure\', \'devices\': [\'PC\', \'XboxSeries\'], \'franchise\': \'Redfall\', \'release date\': \'May 02, 2023\'}, {\'title\': \'Doom Eternal\', \'tags\': [\'First Person Perspective\', \'Realtime Combat\', \'Weapons Combat\', \'Close Combat\', \'Ranged Combat\', \'High Difficulty\', \'Singleplayer\', \'Extreme Difficulty\', \'3D\', \'Branching Skill Progression\', \'Action\', \'Gated Progression\', \'Action - Science Fiction\', \'Realtime\', \'Mastery - Timing and Precision\'], \'publisher\': \'Xbox Game Studios\', \'developer\': \'id Software\', \'primary genre\': \'Shooter\', \'subgenre\': \'1st Person Shooter\', \'devices\': [\'PC\'], \'franchise\': \'Doom\', \'release date\': \'December 01, 2020\'}, {\'title\': \'Cocoon\', \'tags\': [\'Isometric\', \'Indirect Combat\', \'Singleplayer\', \'Branching Skill Progression\', \'Exploration\', \'Magic\', \'Puzzle\', \'Crafting\', \'Realtime\', \'Collecting\', \'Science Fiction - Cyberpunk\', \'Abstract\', \'Stylized Art\', \'Science Fiction - Time Travel\'], \'publisher\': \'Annapurna Interactive\', \'developer\': \'Geometric Interactive\', \'primary genre\': \'Action and Adventure\', \'subgenre\': \'Adventure Puzzle\', \'devices\': [\'PC\', \'XboxOne\', \'XboxSeries\'], \'franchise\': \'Cocoon\', \'release date\': \'September 29, 2023\'}, {\'title\': \'Scorn\', \'tags\': [\'First Person Perspective\', \'Survival Horror\', \'Realtime Combat\', \'Psychological Horror\', \'Alien World\', \'Horror\', \'Exploration\', \'Singleplayer\', \'Atmospheric\', \'Science Fiction\', \'Dark\', \'Science Fiction - Dystopian\', \'Abstract\', \'Action\'], \'publisher\': \'Kepler Interactive\', \'developer\': \'Ebb Software\', \'primary genre\': \'Shooter\', \'subgenre\': \'1st Person Shooter\', \'devices\': [\'PC\', \'XboxSeries\'], \'franchise\': \'Scorn\', \'release date\': \'October 14, 2022\'}]}'}, {'role': 'assistant', 'content': '```json\n{"name": "localdb_query", "arguments": {"query": "Resident Evil 2"}}\n```\nThe function to retrieve the requested attributes for the game "Resident Evil 2" has been called. Please wait for the results to be provided in the next iteration.'}, {'role': 'user', 'content': 'Now use the asking format return your answer again.Return the values in JSON format. '}, {'role': 'assistant', 'content': '{"Tags": "Survival, Action, Horror, Zombies, Remake, Third Person, Gunplay, Puzzle, Sci-Fi, Atmospheric, Gore, Classic, Remastered, Remake", "Primary Genre": "Action and Adventure", "Sub Genre": "Survival Horror", "Devices": "PC, XboxOne, XboxSeries", "Publisher": "Capcom", "Developer": "Capcom", "Franchise": "Resident Evil", "Release Date": "January 25, 2019"}'}]"""
tool_response = json_repair.loads(conttt)
tool_response
# if 'content' in tool_response:
#     for result in tool_response['content']:
#         if 'title' in result:
#             titles.add(result['title'])  # 将 title 添加到集合中


[[{'title': 'ReCore',
   'tags': ['Action',
    'Action - Science Fiction',
    'Third Person Perspective',
    'Realtime Combat',
    'Post-apocalyptic',
    'Singleplayer',
    'Exploration',
    'Platformer',
    'Stylized Art',
    'Science Fiction',
    'Alien World',
    'Weapons Combat',
    'Ranged Combat',
    'Branching Skill Progression',
    'Collecting'],
   'publisher': 'Xbox Game Studios',
   'developer': 'Comcept, Armature Studio',
   'primary genre': 'Platformer',
   'subgenre': 'Action and Adventure',
   'devices': ['PC', 'XboxOne', 'XboxSeries'],
   'franchise': 'ReCore',
   'release date': 'September 13, 2016'},
  {'title': 'Spiritfarer',
   'tags': ['Resource Management',
    'Crafting',
    'Exploration',
    'Simulation',
    'Emotional',
    'Simulation - Life and Romance',
    'Cozy',
    'Hand Drawn Art',
    'Stylized Art',
    'Singleplayer',
    'Character Creation',
    'Atmospheric',
    'Multiplayer Local Cooperative',
    'Side Scroll Perspective',
    